In [8]:
from pyhdf.SD import SD, SDC

# Path to the HDF4 file
file_path = 'G:/Hangkai/Antarctica_MODIS_DATA/2001_2002_Organized_by_day/2001_237/MCD43A4.A2001237.h13v14.061.2020084163912.hdf'

# Open the file
hdf = SD(file_path, SDC.READ)

# Print datasets and their attributes
datasets = hdf.datasets()
for name, info in datasets.items():
    print(name, info)


# Accessing the bands directly by name
nir_dataset = hdf.select('Nadir_Reflectance_Band2')  # NIR band
red_dataset = hdf.select('Nadir_Reflectance_Band1')  # Red band

# Get data as numpy arrays
nir_data = nir_dataset.get()
red_data = red_dataset.get()

# Close the datasets
nir_dataset.endaccess()
red_dataset.endaccess()

# Close the file
hdf.end()

# Here you would typically follow with NDVI calculation as previously described


BRDF_Albedo_Band_Mandatory_Quality_Band1 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 21, 0)
BRDF_Albedo_Band_Mandatory_Quality_Band2 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 21, 1)
BRDF_Albedo_Band_Mandatory_Quality_Band3 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 21, 2)
BRDF_Albedo_Band_Mandatory_Quality_Band4 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 21, 3)
BRDF_Albedo_Band_Mandatory_Quality_Band5 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 21, 4)
BRDF_Albedo_Band_Mandatory_Quality_Band6 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 21, 5)
BRDF_Albedo_Band_Mandatory_Quality_Band7 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 21, 6)
Nadir_Reflectance_Band1 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 22, 7)
Nadir_Reflectance_Band2 (('YDim:MOD_Grid_BRDF', 'XDim:MOD_Grid_BRDF'), (2400, 2400), 22, 8)
Nadir_Reflectance_Band3 (('YDim:MOD_Grid_BRDF', 'XDim

In [21]:
import re
from pyhdf.SD import SD, SDC
import numpy as np
import rasterio
from affine import Affine
import os

def parse_struct_metadata(metadata):
    """Parse the struct metadata to extract geospatial data."""
    pattern = re.compile(r'UpperLeftPointMtrs=\(([^)]+)\)\n\t\tLowerRightMtrs=\(([^)]+)\)')
    match = pattern.search(metadata)
    upper_left = tuple(map(float, match.group(1).split(',')))
    lower_right = tuple(map(float, match.group(2).split(',')))

    return upper_left, lower_right

def calculate_affine_transform(upper_left, lower_right, num_pixels_x=2400, num_pixels_y=2400):
    """Calculate the affine transformation based on corner coordinates."""
    pixel_size_x = (lower_right[0] - upper_left[0]) / num_pixels_x
    pixel_size_y = (upper_left[1] - lower_right[1]) / num_pixels_y
    transform = Affine.translation(upper_left[0], upper_left[1]) * Affine.scale(pixel_size_x, -pixel_size_y)
    return transform

def calculate_ndvi(nir_band, red_band):
    """Calculate NDVI from NIR and Red bands."""
    with np.errstate(divide='ignore', invalid='ignore'):
        ndvi = (nir_band.astype(float) - red_band.astype(float)) / (nir_band + red_band)
        ndvi[np.isnan(ndvi)] = -999  # Set NaN values to a specific nodata value
    return ndvi

day_dir = 'G:\\Hangkai\\Antarctica_MODIS_DATA\\2001_2002_Organized_by_day\\2001_237'

# Loop over each HDF file in the directory
for filename in os.listdir(day_dir):
    if filename.endswith('.hdf'):
        file_path = os.path.join(day_dir, filename)
        hdf = SD(file_path, SDC.READ)

        metadata = hdf.attributes()['StructMetadata.0']
        upper_left, lower_right = parse_struct_metadata(metadata)
        affine_transform = calculate_affine_transform(upper_left, lower_right)

        nir_band = hdf.select('Nadir_Reflectance_Band2').get()
        red_band = hdf.select('Nadir_Reflectance_Band1').get()
        
        ndvi = calculate_ndvi(nir_band, red_band)

        ndvi_meta = {
            'driver': 'GTiff',
            'dtype': 'float32',
            'nodata': -999,
            'width': 2400,
            'height': 2400,
            'count': 1,
            'crs': '+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs',
            'transform': affine_transform
        }

        ndvi_path = file_path.replace('.hdf', '_ndvi.tif')
        with rasterio.open(ndvi_path, 'w', **ndvi_meta) as dst:
            dst.write(ndvi, 1)

        hdf.end()

In [20]:
from pyhdf.SD import SD, SDC
import h5py

def print_structure(hdf_file):
    """Prints the structure of the HDF file, including datasets, groups, and attributes."""
    try:
        # Try using pyhdf to explore SD datasets
        hdf = SD(hdf_file, SDC.READ)
        print("Datasets and attributes using pyhdf:")
        datasets = hdf.datasets()
        for name, info in datasets.items():
            print(f"Dataset: {name}")
            ds = hdf.select(name)
            for attr in ds.attributes().items():
                print(f"  Attr: {attr}")
        # Try to list all global attributes
        print("\nGlobal attributes:")
        for attr in hdf.attributes().items():
            print(f"  {attr}")
    except Exception as e:
        print("Error reading with pyhdf:", e)
    
    try:
        # Use h5py to explore the file structure deeper, including groups
        print("\nStructure and attributes using h5py:")
        with h5py.File(hdf_file, 'r') as file:
            def print_attrs(name, obj):
                print(f"\n{name}")
                for key, val in obj.attrs.items():
                    print(f"  Attr: {key} = {val}")

            file.visititems(print_attrs)
    except Exception as e:
        print("Error reading with h5py:", e)

# Example file path
file_path = 'G:/Hangkai/Antarctica_MODIS_DATA/2001_2002_Organized_by_day/2001_237/MCD43A4.A2001237.h14v16.061.2020084154937.hdf'
print_structure(file_path)



Datasets and attributes using pyhdf:
Dataset: BRDF_Albedo_Band_Mandatory_Quality_Band1
  Attr: ('long_name', 'BRDF_Albedo_Band_Mandatory_Quality_Band1')
  Attr: ('units', 'concatenated flags')
  Attr: ('valid_range', [0, 254])
  Attr: ('_FillValue', 255)
  Attr: ('Description', 'Mandatory QA:\n  0 = processed, good quality (full BRDF inversions)\n  1 = processed, see other QA (magnitude BRDF inversions)\n')
Dataset: BRDF_Albedo_Band_Mandatory_Quality_Band2
  Attr: ('long_name', 'BRDF_Albedo_Band_Mandatory_Quality_Band2')
  Attr: ('units', 'concatenated flags')
  Attr: ('valid_range', [0, 254])
  Attr: ('_FillValue', 255)
  Attr: ('Description', 'Mandatory QA:\n  0 = processed, good quality (full BRDF inversions)\n  1 = processed, see other QA (magnitude BRDF inversions)\n')
Dataset: BRDF_Albedo_Band_Mandatory_Quality_Band3
  Attr: ('long_name', 'BRDF_Albedo_Band_Mandatory_Quality_Band3')
  Attr: ('units', 'concatenated flags')
  Attr: ('valid_range', [0, 254])
  Attr: ('_FillValue', 25

In [22]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os

def mosaic_ndvi_files(directory, output_file):
    """Create a mosaic from NDVI TIFF files in a given directory."""
    # Search for TIFF files in the directory
    search_criteria = "*.tif"
    q = os.path.join(directory, search_criteria)
    tif_files = glob.glob(q)
    
    # Open all the TIFF files
    src_files_to_mosaic = [rasterio.open(fp) for fp in tif_files]
    
    # Merge the files
    mosaic, out_trans = merge(src_files_to_mosaic)
    
    # Copy the metadata
    out_meta = src_files_to_mosaic[0].meta.copy()
    
    # Update the metadata to reflect the number of layers
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "crs": src_files_to_mosaic[0].crs
    })
    
    # Write the mosaic raster to disk
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)
    
    # Close all raster files
    for src in src_files_to_mosaic:
        src.close()
    
    return output_file

# Directory containing the NDVI TIFF files
ndvi_directory = 'G:\\Hangkai\\Antarctica_MODIS_DATA\\2001_2002_Organized_by_day\\2001_237'
output_mosaic_file = 'G:\\Hangkai\\Antarctica_MODIS_DATA\\2001_2002_Organized_by_day\\ndvi_mosaic.tif'

# Run the mosaicking function
mosaic_file = mosaic_ndvi_files(ndvi_directory, output_mosaic_file)
print(f"Mosaic created: {mosaic_file}")

Mosaic created: G:\Hangkai\Antarctica_MODIS_DATA\2001_2002_Organized_by_day\ndvi_mosaic.tif
